In [1]:
import requests, json
import pandas as pd

from model.post import Post

In [2]:
def get_post(keyword: str, num_page: int):
    url = "https://www.grants.gov/grantsws/rest/opportunities/search/"
    headers = {
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Content-Length": "143",
        "Content-Type": "application/json; charset=UTF-8",
        "Cookie": "JSESSIONID=76CDCB5F1830D1CDB087A3A03E97BD7A; BIGipServerProd-Liferay-HTTPS-Pool=3473001738.32287.0000; BIGipServerProd-Apache-HTTP-Pool=3426864394.3367.0000; LFR_SESSION_STATE_5=1684661798222",
        "Host": "www.grants.gov",
        "Origin": "https://www.grants.gov",
        "Referer": "https://www.grants.gov/custom/search.jsp",
        "sec-ch-ua": '"Microsoft Edge";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
        "sec-ch-ua-platform": "Windows",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.50",
        "X-Requested-With": "XMLHttpRequest"
    }
    payload = {
        "cfda":"",
        "eligibilities":"23",
        "keyword":keyword,
        "oppNum":"",
        "oppStatuses":"forecasted|posted",
        "sortBy":"oppName|desc",
        "startRecordNum": num_page
    }
    data = requests.post(url=url,headers=headers,json=payload)
    if data.status_code != 200:
        raise ValueError("Request data fail!",data.json())
    return data


def get_all_post(keyword: str):
    num_page = 0
    datas = []
    while True:
        data = get_post(keyword,num_page).json()['oppHits']
        datas += data
        if len(data) < 25 or data == []:
            break
        num_page += 25
    return datas

def get_detail_post(id_post: int):
    cookies = {
        'BIGipServerProd-Liferay-HTTPS-Pool': '3473001738.32287.0000',
        'LFR_SESSION_STATE_5': '1684666108784',
        'JSESSIONID': '9D8839E80CB1B729696FCC8D12A25B5F',
        'BIGipServerProd-Apache-HTTP-Pool': '2483145994.3367.0000',
    }

    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        # 'Cookie': 'BIGipServerProd-Liferay-HTTPS-Pool=3473001738.32287.0000; LFR_SESSION_STATE_5=1684666108784; JSESSIONID=9D8839E80CB1B729696FCC8D12A25B5F; BIGipServerProd-Apache-HTTP-Pool=2483145994.3367.0000',
        'Origin': 'https://www.grants.gov',
        'Referer': f'https://www.grants.gov/custom/viewOppDetails.jsp?oppId={id_post}',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.50',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua': '"Microsoft Edge";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    data = {
        'oppId': id_post,
    }

    response = requests.post('https://www.grants.gov/grantsws/rest/opportunity/details', cookies=cookies, headers=headers, data=data)
    if response.status_code != 200:
        raise ValueError("Request data fail!",data.json())
    return response

In [16]:
keyword = 'education'
datas = get_all_post(keyword)

In [3]:
# df = pd.DataFrame(datas)
# df.to_csv(f'../data/{keyword}.csv')
df = pd.read_csv(f'data/education.csv')

In [5]:
posts = []
for idx, row in df.iterrows():
    d_post = row.to_dict()
    d_data =get_detail_post(int(row['id'])).json()
    
    post = Post()
    post.convert_structure(d_post,d_data)
    post_detail = post.__dict__
    posts.append(post_detail)
    break

In [7]:
pd.DataFrame(posts).to_csv('data/post_education.csv',index=False)